In [245]:
import os
import glob as mark
import pandas as pd
from scipy.stats import linregress

In [246]:
data_list = mark.glob(".\\data\\*.csv")
data = {}
for file in data_list:
    key = file.split("\\")[-1].replace(".csv", "")
    df = pd.read_csv(file, encoding="latin-1")
    data[key] = df

print( data.keys() )

dict_keys(['Cities', 'Conferences', 'MConferenceTourneyGames', 'MGameCities', 'MMasseyOrdinals', 'MNCAATourneyCompactResults', 'MNCAATourneyDetailedResults', 'MNCAATourneySeedRoundSlots', 'MNCAATourneySeeds', 'MNCAATourneySlots', 'MRegularSeasonCompactResults', 'MRegularSeasonDetailedResults', 'MSeasons', 'MSecondaryTourneyCompactResults', 'MSecondaryTourneyTeams', 'MTeamCoaches', 'MTeamConferences', 'MTeams', 'MTeamSpellings', 'SampleSubmission2023', 'WGameCities', 'WNCAATourneyCompactResults', 'WNCAATourneyDetailedResults', 'WNCAATourneySeeds', 'WNCAATourneySlots', 'WRegularSeasonCompactResults', 'WRegularSeasonDetailedResults', 'WSeasons', 'WTeamConferences', 'WTeams', 'WTeamSpellings'])


In [247]:
sample_sub = data["SampleSubmission2023"].copy()
regular_season = pd.concat([data["MRegularSeasonDetailedResults"], data["WRegularSeasonDetailedResults"]]).copy()
tourney_results = pd.concat([data["MNCAATourneyCompactResults"], data["WNCAATourneyCompactResults"]]).copy()
massey_ordinals = data["MMasseyOrdinals"].copy()

print( "done." )

done.


In [248]:
massey_ordinals.columns

Index(['Season', 'RankingDayNum', 'SystemName', 'TeamID', 'OrdinalRank'], dtype='object')

In [249]:
agg = {"TeamID": "count"}
top_rank_systems = (
    massey_ordinals.groupby(["SystemName"])
    .agg(agg)
    .reset_index()
    .rename({"TeamID": "Count"}, axis=1)
    .sort_values(by="Count", ascending=False)
    .head(5)
).SystemName.tolist()

top_rank_systems

['SAG', 'MOR', 'POM', 'DOK', 'WLK']

In [250]:
top_ordinals = massey_ordinals[massey_ordinals["SystemName"].isin(top_rank_systems)].copy().reset_index().drop("index", axis=1)
top_ordinals.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,37,MOR,1102,110
1,2003,37,MOR,1103,179
2,2003,37,MOR,1104,13
3,2003,37,MOR,1105,262
4,2003,37,MOR,1106,272


In [251]:
pvt = top_ordinals.pivot(index=["Season", "RankingDayNum", "TeamID"], columns="SystemName", values="OrdinalRank").reset_index().reset_index()
if "SystemName" in pvt.columns:
    pvt.drop("SystemName", axis=1, inplace=True)
pvt.isnull().sum()

SystemName
index                0
Season               0
RankingDayNum        0
TeamID               0
DOK              30433
MOR              14646
POM              17137
SAG              13656
WLK              39745
dtype: int64

In [252]:
sort_cols = ["TeamID", "Season", "RankingDayNum"]
pvt.sort_values(by=sort_cols, inplace=True)

In [253]:
season = 2014
team = 1101
mask = (pvt["Season"]==season) & (pvt["TeamID"]==team)
pvt[mask]

SystemName,index,Season,RankingDayNum,TeamID,DOK,MOR,POM,SAG,WLK
80355,80355,2014,9,1101,334.0,348.0,336.0,349.0,NaN
80706,80706,2014,16,1101,346.0,348.0,340.0,350.0,NaN
81057,81057,2014,23,1101,342.0,348.0,331.0,349.0,NaN
81408,81408,2014,30,1101,345.0,349.0,338.0,347.0,336.0
81759,81759,2014,37,1101,346.0,350.0,337.0,347.0,335.0
82110,82110,2014,44,1101,346.0,349.0,334.0,347.0,332.0
82461,82461,2014,51,1101,343.0,349.0,335.0,341.0,315.0
82812,82812,2014,58,1101,337.0,350.0,335.0,335.0,314.0
83163,83163,2014,65,1101,324.0,350.0,336.0,329.0,310.0
83514,83514,2014,72,1101,327.0,349.0,338.0,342.0,316.0


In [254]:
pvt["POM"].notnull()

80355     True
80706     True
81057     True
81408     True
81759     True
          ... 
141485    True
141848    True
142211    True
142574    True
142937    True
Name: POM, Length: 142938, dtype: bool

In [255]:
teams = pvt.TeamID.unique()
output = []
for team in teams:
    tdf = pvt[pvt["TeamID"]==team].copy()
    seasons = tdf.Season.unique()
    for season in seasons:
        tdf_s = tdf[tdf["Season"]==season].copy()
        for ranking in top_rank_systems:
            tdf_r = tdf_s[tdf_s[ranking].notnull()].copy()
            ranks = tdf_r[ranking].values
            if len(ranks)<2:
                continue
            initial = ranks[0]
            final = ranks[-1]
            change = (final - initial) / initial
            output.append([team, season, ranking, initial, final, change])
            

In [256]:
output_df = pd.DataFrame(output, columns=["team", "season", "ranking", "initial", "final", "change"])
print( output_df.shape )
output_df.head(10)

(35321, 6)


,team,season,ranking,initial,final,change
0,1101,2014,SAG,349.0,346.0,-0.008596
1,1101,2014,MOR,348.0,349.0,0.002874
2,1101,2014,POM,336.0,348.0,0.035714
3,1101,2014,DOK,334.0,346.0,0.035928
4,1101,2014,WLK,336.0,330.0,-0.017857
5,1101,2015,SAG,314.0,336.0,0.070064
6,1101,2015,MOR,303.0,346.0,0.141914
7,1101,2015,POM,349.0,332.0,-0.048711
8,1101,2015,DOK,336.0,341.0,0.014881
9,1101,2015,WLK,335.0,332.0,-0.008955


In [257]:
output_df[output_df["change"]<-0.8]

,team,season,ranking,initial,final,change
342,1104,2021,MOR,42.0,7.0,-0.833333
343,1104,2021,POM,47.0,8.0,-0.829787
344,1104,2021,DOK,39.0,4.0,-0.897436
351,1104,2023,SAG,15.0,1.0,-0.933333
353,1104,2023,POM,15.0,2.0,-0.866667
...,...,...,...,...,...,...
34476,1458,2010,MOR,31.0,5.0,-0.838710
34477,1458,2010,POM,37.0,3.0,-0.918919
34520,1458,2019,SAG,100.0,16.0,-0.840000
34623,1459,2019,MOR,71.0,14.0,-0.802817


In [258]:
output_pvt = output_df.pivot(index=["team", "season"], columns="ranking", values=["final", "change"]).reset_index().reset_index()
output_pvt.columns = output_pvt.columns.to_flat_index()
output_pvt.columns = ['_'.join(col) if len(col[1])>0 else col[0] for col in output_pvt.columns.values]
output_pvt.drop("index", axis=1, inplace=True)

output_pvt.head()

,team,season,final_DOK,final_MOR,final_POM,final_SAG,final_WLK,change_DOK,change_MOR,change_POM,change_SAG,change_WLK
0,1101,2014,346.0,349.0,348.0,346.0,330.0,0.035928,0.002874,0.035714,-0.008596,-0.017857
1,1101,2015,341.0,346.0,332.0,336.0,332.0,0.014881,0.141914,-0.048711,0.070064,-0.008955
2,1101,2016,314.0,311.0,318.0,320.0,304.0,-0.084548,-0.063253,-0.080925,-0.077810,-0.070336
3,1101,2017,320.0,317.0,300.0,305.0,307.0,-0.061584,0.025890,-0.074074,-0.016129,0.497561
4,1101,2018,263.0,265.0,262.0,276.0,261.0,-0.180685,0.090535,0.003831,-0.098039,-0.029740


In [259]:
new_col_names = {
    "team": "TeamID",
    "season": "Season"
}

output_pvt.rename(new_col_names, axis=1, inplace=True)
output_pvt

,TeamID,Season,final_DOK,final_MOR,final_POM,final_SAG,final_WLK,change_DOK,change_MOR,change_POM,change_SAG,change_WLK
0,1101,2014,346.0,349.0,348.0,346.0,330.0,0.035928,0.002874,0.035714,-0.008596,-0.017857
1,1101,2015,341.0,346.0,332.0,336.0,332.0,0.014881,0.141914,-0.048711,0.070064,-0.008955
2,1101,2016,314.0,311.0,318.0,320.0,304.0,-0.084548,-0.063253,-0.080925,-0.077810,-0.070336
3,1101,2017,320.0,317.0,300.0,305.0,307.0,-0.061584,0.025890,-0.074074,-0.016129,0.497561
4,1101,2018,263.0,265.0,262.0,276.0,261.0,-0.180685,0.090535,0.003831,-0.098039,-0.029740
...,...,...,...,...,...,...,...,...,...,...,...,...
7259,1473,2023,349.0,345.0,345.0,356.0,342.0,0.175084,-0.022663,-0.011461,-0.016575,0.082278
7260,1474,2023,220.0,202.0,218.0,266.0,198.0,-0.214286,0.980392,-0.026786,-0.254902,0.245283
7261,1475,2023,267.0,272.0,267.0,314.0,260.0,0.608434,0.402062,0.335000,-0.079179,0.293532
7262,1476,2023,326.0,325.0,332.0,335.0,316.0,0.288538,-0.094708,-0.034884,-0.058989,0.183521


In [ ]:
output_pvt.to_csv("./output/massey_ordinals.csv", index=None)